# Removendo Dados Duplicados e Esparsos

A tarefa mais básica na limpeza de dados é detectar e remover dados errôneos. Isso inclui dados duplicados e dados ausentes ou não confiáveis. Esta não é a tarefa mais glamorosa, mas é extremamente importante. Como diz o velho ditado, "entra lixo, sai lixo". Ser capaz de organizar e limpar conjuntos de dados desorganizados é absolutamente essencial para o sucesso e pode diferenciá-lo de outros na área de ciência/engenharia de dados.

Para começar, crie este dataframe de dados meteorológicos.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id_registro" : ['DCMXP87EDE', 'DCMXP87EDE', 'ZMIFM3HX9G', 'HIVVXBAPS2', 'U1AA66UDES', 'B20KL5PW3L', 'FIZLY34KSQ'],
    "polegadas_chuva" : [1.1, 1.1, 0.0, 0.0, 2.4, 11.2, 3.2],
    "tornado" : [0, 0, 1, 0, 0, 0, 0],
    "raios" : [0, 0, 1, 1, 1, 0, 0],
    "velocidade_vento_mph" : [3.1, 3.1, 143.0, None, 8.1, 5.0, None],
    "gravidade" : ['LIMPO', 'LIMPO', 'SEVERO', 'LEVE', 'LEVE', 'GRAVE', None],
    "indicador_transmissao" : [1, 1, 1, 1, 1, 1, 1]
})

df


## De onde vieram os dados?

Você pode se sentir tentado a começar a escrever código Python e a manipular conjuntos de dados em dataframes do Pandas, mas vamos recuar um pouco e fazer algumas perguntas. De onde vieram esses dados? Como foram coletados? Quais sensores ou métodos de entrada de dados foram usados ​​para coletá-los? Os dados podem estar enviesados ​​de alguma forma ou com variáveis ​​importantes ausentes?

É igualmente importante, se não mais, perguntar não apenas o que os dados dizem, mas também de onde eles vieram. Isso pode revelar problemas maiores que estão contaminando seus dados, mas não são detectáveis ​​apenas observando o conjunto de dados. Os dados podem estar enviesados ​​ou com dados ou variáveis ​​relevantes ausentes para o problema a ser resolvido. Se você tiver dados cheios de valores vazios (para os quais discutiremos técnicas de remoção), você deve entender completamente por que eles estão vazios e se há um problema mais profundo no processo de produção dos dados. Por exemplo, se um sensor de temperatura quebrado estiver registrando valores `NA` ou `NaN` em uma estação meteorológica específica, você deve considerar consertar esse sensor em vez de simplesmente remover esses registros completamente. Se uma estação estiver produzindo registros duplicados, o bug do software deve ser corrigido em vez de remover as duplicatas.

Há algumas coisas que você não pode quantificar ou aplicar uma função Pandas para corrigir, e você deve aplicar julgamento qualitativo para fazer as perguntas certas e abordar os problemas na fonte. Depois de esgotar essas perguntas e compreender completamente seu conjunto de dados, você pode prosseguir adequadamente.

## Removendo linhas duplicadas

Vamos imprimir nosso dataframe de dados meteorológicos.

In [ ]:
df

Observe acima como temos alguns dados questionáveis, incluindo as duas primeiras linhas sendo duplicadas e alguns valores `NaN` e `None` ausentes. Vamos nos concentrar primeiro nas duplicatas.

Para obter todas as duplicatas, exceto a primeira instância de uma linha, use a função `duplicated()`.

In [ ]:
df.duplicated()

Você pode sinalizar todas as instâncias (incluindo a primeira instância encontrada) definindo `keep=False`.

In [ ]:
df.duplicated(keep=False)

Se quiser encontrar duplicatas com base apenas em uma ou mais colunas como chave, use a função `subset()`. Abaixo, encontramos registros duplicados usando apenas o campo `id_registro`.

In [ ]:
df.duplicated(subset=['id_registro'])

Poderíamos compor nossa condição com vários campos, se quiséssemos, como `id_registro` e `polegadas_chuva`.

In [ ]:
df.duplicated(subset=['id_registro','polegadas_chuva'])

Poderíamos usar o booleano `Series` retornado nos exemplos acima para extrair apenas essas linhas para um novo dataframe. No entanto, também podemos usar a função `drop_duplicates()` para fazer isso. Ela aceita os mesmos argumentos que `duplicatated()` e possui um parâmetro `inplace` para substituir o dataframe existente.

In [ ]:
df.drop_duplicates(inplace=True)

E, claro, você sempre pode descartar com base em um subconjunto.

In [ ]:
df.drop_duplicates(subset=['id_registro'], inplace=True)
df

> Observe que existem funções quase idênticas para lidar com duplicatas para [Index](https://pandas.pydata.org/docs/reference/api/pandas.Index.duplicated.html#pandas.Index.duplicated) e [Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.duplicated.html#pandas.Series.duplicated), também chamadas de `duplicated()` e `drop_duplicates()`. Elas operam de forma muito semelhante à contraparte de dataframe para essas funções.

## Remover colunas com um valor

Colunas com um único valor provavelmente não serão úteis para aprendizado de máquina e outras análises. Portanto, são candidatas à remoção, desde que isso não seja um erro. Observe como `indicador_transmissao` é todo 1, o que não ajuda em nada.

In [ ]:
df

Podemos usar a função `nunique()` para identificar o número de valores únicos em cada coluna como uma série.

In [ ]:
df.nunique()

Podemos iterar a série acima e rastrear quais índices de coluna excluir, com base no fato de eles terem apenas um valor exclusivo.

In [ ]:
# identificar colunas de valor único a serem excluídas
deletar_colunas = [c for c,v in zip(df.columns, df.nunique()) if v == 1]
deletar_colunas

Por fim, podemos remover essas colunas (neste caso, haverá apenas uma) passando-as para a função drop. Certifique-se de especificar que estamos removendo colunas, especificando `axis=1`.

In [ ]:
df.drop(deletar_colunas, axis=1, inplace=True)
df

## Remover colunas com poucos valores

Ao lidar com valores categóricos, não é de se surpreender que existam poucos valores. Em nossos dados meteorológicos, esperamos apenas um `True` ou `False` para campos booleanos. Esperamos apenas cerca de 4 valores possíveis para `gravidade`, como `GRAVE`, `LEVE`, `LIMPO` e `SEVERO`. Raramente devemos considerar variáveis ​​discretas como essa como muito esparsas para serem usadas.

No entanto, lidar com valores numéricos/contínuos (decimais) é uma história diferente. Quando uma coluna numérica tem poucos valores, pode não haver muita variância para fazer previsões significativas do modelo. Se esse for realmente o caso, eles devem ser removidos. Isso nem sempre acontece, portanto, certifique-se de removê-los quando houver absoluta certeza de que eles não agregam valor. Às vezes, a escolha do modelo também impactará essa decisão, pois modelos lineares frequentemente dependem de alguma variância para uma distribuição significativa dos dados.

Vamos trazer um conjunto de dados diferente para este exemplo: um conjunto de dados de qualidade do vinho com diferentes atributos do vinho.

In [ ]:
vinho_df = pd.read_csv('https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/winequality-red.csv')
vinho_df

Uma métrica que pode nos guiar para colunas com baixo número de valores únicos é, para cada coluna, a proporção de valores únicos em todas as linhas. Abaixo, pegamos cada coluna e dividimos o número de valores únicos pelo número de linhas.

In [ ]:
n_linhas, n_colunas = vinho_df.shape

for i in range(n_colunas):
    num_unicos = vinho_df.iloc[:, i].nunique()
    percentual = float(num_unicos) / n_linhas * 100
    print(f'{i}, {num_unicos}, {round(percentual, 2)}%')

Como você pode ver acima, existem algumas colunas com porcentagens muito baixas de valores únicos. As categóricas são esperadas, como a última coluna, `quality`. Mas algumas colunas, como `alcohol` (na posição 11) e `free_sulfur_dioxide` (na posição 5), apresentam porcentagens muito baixas.

Digamos que queremos remover colunas com 5% ou menos de valores únicos. Vamos adaptar nosso loop `for` acima para extrair rótulos de colunas que tenham uma porcentagem de valores únicos de `0,05` ou menos.

In [ ]:
colunas_para_excluir = []

n_linhas, n_colunas = vinho_df.shape

for i in range(n_colunas):
    num_unicos = vinho_df.iloc[:, i].nunique()
    percentual = float(num_unicos) / n_linhas  
    if percentual <= 0.05:
        colunas_para_excluir.append(vinho_df.columns[i])

colunas_para_excluir

Em seguida, pegaremos essas três colunas e as removeremos. Você notará que elas foram removidas.

In [ ]:
vinho_df.drop(colunas_para_excluir, axis=1, inplace=True)
vinho_df

## Remover colunas com baixa variância

Outra maneira de abordar esse problema de colunas com poucos valores únicos é calcular a variância e usá-la como um limite de corte. Lembre-se de que a variância $ \sigma^2 $ é uma medida em estatística que calcula a média das diferenças quadradas entre cada valor observado $ x_i $ e a média $ \mu $ desses valores. Em outras palavras, para calcular a variância ao quadrado, a diferença entre cada ponto de dados $ x_i $ e a média $ \mu $, some-os e divida pelo número de elementos $ n $.

$$
\sigma^2 = \frac{\sum_{i=1}^{n}(x_i - \mu)^2} {n}
$$ 

Vamos carregar nosso conjunto de dados de vinhos novamente para recomeçar e trazer de volta as colunas removidas.

In [ ]:
vinho_df = pd.read_csv('https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/winequality-red.csv')
vinho_df

Quanto menor a variância, menos valores únicos podemos esperar. Existe um utilitário útil, `VarianceThreshold`, no scikit-learn, que pode ser usado para remover atributos com base na variância. Normalmente, queremos mais variância para fins de modelagem em estatística e aprendizado de máquina. Ter pouca variância em um atributo não será útil. Vamos declarar uma instância de `VarianceThreshold` aqui e definir seu limite como `.05`. Quanto maior for esse parâmetro, mais colunas ele eliminará devido aos limites mais altos para variância.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=.05)

Em seguida, vamos extrair apenas as colunas da variável de entrada, selecionando todas, exceto a última coluna (que é `quality`). Em seguida, passamos para a função `fit_transform()` do `VariableThreshold` para obter as colunas de dados que atingiram esse limite.

In [ ]:
X = vinho_df.iloc[:,:-1]
X_threshold = vt.fit_transform(X)
X_threshold

Então, quantas colunas passaram e atingiram esse limite de variância? Vamos dar uma olhada no formato e contar o número de colunas antes e depois da transformação.

In [ ]:
print(f"NÚMERO DE VARIÁVEIS ANTES: {X.shape[1]}")
print(f"NÚMERO DE VARIÁVEIS DEPOIS: {X_threshold.shape[1]}")

Portanto, 6 colunas foram eliminadas. Infelizmente, nessa transformação, nosso `DataFrame` foi transformado em um `ndarray` NumPy. Felizmente, existe uma função `get_support()` no `VarianceThreshold` para retornar os índices das colunas que passam pelo limite. Podemos então passar isso de volta para a propriedade `columns` para obter os índices das colunas e, em seguida, usá-los para selecionar essas colunas do nosso dataframe.

In [ ]:
vinho_df[vinho_df.columns[vt.get_support(indices=True)]]

Como você pode ver, todas as colunas, exceto 5, foram eliminadas e não ultrapassaram o limite de variância.

## Exercício

Abaixo está um dataframe de dados do termostato. Complete o código substituindo os pontos de interrogação "?" para remover registros duplicados e quaisquer colunas com 3 ou menos valores exclusivos.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id_registro" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperatura" : [65.2, 65.2, 47.2, 47.2, 57.4, 23.4, 27.5], 
    "umidade" : [.8, .8, .7, .7, .7, .7, .8],
    "estavel" : [True, True, True, True, True, True, True]
})

# remover duplicados
df.?

# remover colunas com 3 ou menos valores únicos
colunas_para_apagar = []

n_linhas, n_colunas = df.shape

for i in range(n_colunas):
    num_unicos = df.iloc[:, i].nunique()
    if num_unicos <= 3:
        colunas_para_apagar.append(df.columns[i])
    
df.?(colunas_para_apagar, axis=?, inplace=True)
df

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id_registro" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperatura" : [65.2, 65.2, 47.2, 47.2, 57.4, 23.4, 27.5], 
    "umidade" : [.8, .8, .7, .7, .7, .7, .8],
    "estavel" : [True, True, True, True, True, True, True]
})

# remover duplicados
df.drop_duplicates(inplace=True)

# remover colunas com 3 ou menos valores únicos
colunas_para_apagar = []

n_linhas, n_colunas = df.shape

for i in range(n_colunas):
    num_unicos = df.iloc[:, i].nunique()
    if num_unicos <= 3:
        colunas_para_apagar.append(df.columns[i])
    
df.drop(colunas_para_apagar, axis=1, inplace=True)
df